In [3]:
pip install folium 

Note: you may need to restart the kernel to use updated packages.


### 지하철 승하차 인원 Heatmap

In [4]:
## 경고 메세지 표시 안하기
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
import pandas as pd
import seaborn as sns
import folium
import matplotlib.pyplot as plt
plt.rc('font', family='malgun gothic')
plt.rcParams['axes.unicode_minus'] = False

In [6]:
data = pd.read_csv("Data/seoul-metro-2021.logs.csv")

In [7]:
data.head()

,timestamp,station_code,people_in,people_out
0,2021-01-01T05:00:00.000+09:00,150,86,85
1,2021-01-01T06:00:00.000+09:00,150,111,355
2,2021-01-01T07:00:00.000+09:00,150,157,438
3,2021-01-01T08:00:00.000+09:00,150,306,592
4,2021-01-01T09:00:00.000+09:00,150,333,841


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941553 entries, 0 to 1941552
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   timestamp     object
 1   station_code  int64 
 2   people_in     int64 
 3   people_out    int64 
dtypes: int64(3), object(1)
memory usage: 59.3+ MB


In [20]:
# 일자별 Station별 승하차 인원 Sum

station_sum = data.groupby("station_code").sum()

In [21]:
station_sum

,timestamp,people_in,people_out
station_code,,,
150,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,12697273,12109991
151,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,5997344,6030491
152,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,9638952,9284693
153,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,7704599,7090896
154,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,6651283,6609055
...,...,...,...
2824,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,3206211,3097722
2825,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,1395919,1495603
2826,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,1614590,1475010


In [10]:
station_info = pd.read_csv("Data/seoul-metro-station-info.csv")

In [11]:
station_info.head()

,station.code,station.fr_code,line.num,line.name,line.name_sub,line.station_seq,station.name_full,station.name,station.name_chc,station.name_chn,station.name_en,station.name_jp,geo.latitude,geo.longitude,geo.sigungu_code,geo.sigungu_name,geo.addres_road,geo.address_land,geo.phone
0,158,124,1,1호선,지하철1호선,1,청량리(서울시립대입구),청량리|서울시립대입구,祭基洞,祭基洞,Jegidong,チェギドン,37.580178,127.046835,11060,동대문구,서울특별시 동대문구 왕산로 지하205(전농동),서울특별시 동대문구 전농동 620-69 청량리역(1호선),02-6110-1241
1,157,125,1,1호선,지하철1호선,2,제기동,제기동,新設洞,新设洞,Sinseoldong,シンソルトン,37.578103,127.034893,11060,동대문구,서울특별시 동대문구 왕산로 지하93(제기동),서울특별시 동대문구 제기동 65 제기동역(1호선),02-6110-1251
2,156,126,1,1호선,지하철1호선,3,신설동,신설동,東廟앞,东庙,Dongmyo,トンミョアプ,37.575297,127.025087,11060,동대문구,서울특별시 동대문구 왕산로 지하1(신설동),서울특별시 동대문구 신설동 76-5 신설동역(1호선),02-6110-1261
3,159,127,1,1호선,지하철1호선,4,동묘앞,동묘앞,淸凉里(서울市立大入口),清凉里(首尔市立大学),Cheongnyangni(University of Seoul),チョンニャンニ,37.572627,127.016429,11010,종로구,서울특별시 종로구 종로 359(숭인동),서울특별시 종로구 숭인동 117 동묘앞역(1호선),02-6110-1271
4,155,128,1,1호선,지하철1호선,5,동대문,동대문,東大門,东大门,Dongdaemun,トンデムン,37.571420,127.009745,11010,종로구,서울특별시 종로구 종로 지하302(창신동),서울특별시 종로구 창신동 492-1 동대문역(1호선),02-6110-1281


In [12]:
station_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   station.code       285 non-null    int64  
 1   station.fr_code    285 non-null    object 
 2   line.num           285 non-null    int64  
 3   line.name          285 non-null    object 
 4   line.name_sub      285 non-null    object 
 5   line.station_seq   285 non-null    int64  
 6   station.name_full  285 non-null    object 
 7   station.name       285 non-null    object 
 8   station.name_chc   285 non-null    object 
 9   station.name_chn   285 non-null    object 
 10  station.name_en    285 non-null    object 
 11  station.name_jp    285 non-null    object 
 12  geo.latitude       285 non-null    float64
 13  geo.longitude      285 non-null    float64
 14  geo.sigungu_code   285 non-null    int64  
 15  geo.sigungu_name   285 non-null    object 
 16  geo.addres_road    285 non

In [15]:
station_info = station_info[['station.code', 'geo.latitude', 'geo.longitude']]

In [16]:
station_info

,station.code,geo.latitude,geo.longitude
0,158,37.580178,127.046835
1,157,37.578103,127.034893
2,156,37.575297,127.025087
3,159,37.572627,127.016429
4,155,37.571420,127.009745
...,...,...,...
280,2823,37.451535,127.159816
281,2824,37.445210,127.156866
282,2825,37.440918,127.147564
283,2826,37.437428,127.140722


In [17]:
station_info = station_info.set_index('station.code')

In [18]:
station_info

,geo.latitude,geo.longitude
station.code,,
158,37.580178,127.046835
157,37.578103,127.034893
156,37.575297,127.025087
159,37.572627,127.016429
155,37.571420,127.009745
...,...,...
2823,37.451535,127.159816
2824,37.445210,127.156866
2825,37.440918,127.147564


In [22]:
joined_data = station_sum.join(station_info)

In [23]:
joined_data

,timestamp,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,,
150,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,12697273,12109991,37.554648,126.972559
151,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,5997344,6030491,37.564718,126.977108
152,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,9638952,9284693,37.570161,126.982923
153,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,7704599,7090896,37.571607,126.991806
154,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,6651283,6609055,37.570926,127.001849
...,...,...,...,...,...
2824,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,3206211,3097722,37.445210,127.156866
2825,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,1395919,1495603,37.440918,127.147564
2826,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,1614590,1475010,37.437428,127.140722


In [25]:
joined_data = joined_data[['people_in', 'people_out', 'geo.latitude', 'geo.longitude']]

In [26]:
joined_data

,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,
150,12697273,12109991,37.554648,126.972559
151,5997344,6030491,37.564718,126.977108
152,9638952,9284693,37.570161,126.982923
153,7704599,7090896,37.571607,126.991806
154,6651283,6609055,37.570926,127.001849
...,...,...,...,...
2824,3206211,3097722,37.445210,127.156866
2825,1395919,1495603,37.440918,127.147564
2826,1614590,1475010,37.437428,127.140722


In [29]:
seoul_in = folium.Map(location = [37.55, 126.98], zoom_start = 12)

In [34]:
seoul_in

In [31]:
from folium.plugins import HeatMap

In [32]:
HeatMap ( data = joined_data [['geo.latitude', 'geo.longitude', 'people_in']]).add_to(seoul_in)

In [36]:
seoul_in

In [ ]:
seoul_in.save('seoul_in.html')

In [37]:
HeatMap ( data = joined_data [['geo.latitude', 'geo.longitude', 'people_out']]).add_to(seoul_in)
seoul_in

In [38]:
seoul_in.save('seoul_out.html')

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941553 entries, 0 to 1941552
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   timestamp     object
 1   station_code  int64 
 2   people_in     int64 
 3   people_out    int64 
dtypes: int64(3), object(1)
memory usage: 59.3+ MB


### 출퇴근시간대의 Heatmap

In [43]:
morning_data = data[(pd.to_datetime(data.timestamp).dt.hour > 5) & (pd.to_datetime(data.timestamp).dt.hour < 9)]

In [44]:
morning_data

,timestamp,station_code,people_in,people_out
1,2021-01-01T06:00:00.000+09:00,150,111,355
2,2021-01-01T07:00:00.000+09:00,150,157,438
3,2021-01-01T08:00:00.000+09:00,150,306,592
20,2021-01-01T06:00:00.000+09:00,151,45,145
21,2021-01-01T07:00:00.000+09:00,151,75,176
...,...,...,...,...
1941517,2021-12-31T07:00:00.000+09:00,2827,279,120
1941518,2021-12-31T08:00:00.000+09:00,2827,391,311
1941535,2021-12-31T06:00:00.000+09:00,2828,83,41
1941536,2021-12-31T07:00:00.000+09:00,2828,279,119


In [45]:
morning_station_sum = morning_data.groupby("station_code").sum()

In [46]:
morning_station_sum

,timestamp,people_in,people_out
station_code,,,
150,2021-01-01T06:00:00.000+09:002021-01-01T07:00:...,1173032,3383130
151,2021-01-01T06:00:00.000+09:002021-01-01T07:00:...,192838,2694129
152,2021-01-01T06:00:00.000+09:002021-01-01T07:00:...,193164,3595975
153,2021-01-01T06:00:00.000+09:002021-01-01T07:00:...,179542,1182753
154,2021-01-01T06:00:00.000+09:002021-01-01T07:00:...,165788,1733838
...,...,...,...
2824,2021-01-01T06:00:00.000+09:002021-01-01T07:00:...,1037376,434322
2825,2021-01-01T06:00:00.000+09:002021-01-01T07:00:...,372556,156490
2826,2021-01-01T06:00:00.000+09:002021-01-01T07:00:...,499319,168571


In [47]:
morning_station_sum = morning_station_sum[['people_in', 'people_out']]

In [48]:
morning_station_sum

,people_in,people_out
station_code,,
150,1173032,3383130
151,192838,2694129
152,193164,3595975
153,179542,1182753
154,165788,1733838
...,...,...
2824,1037376,434322
2825,372556,156490
2826,499319,168571


In [49]:
morning_joined_data = morning_station_sum.join(station_info)

In [50]:
morning_joined_data

,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,
150,1173032,3383130,37.554648,126.972559
151,192838,2694129,37.564718,126.977108
152,193164,3595975,37.570161,126.982923
153,179542,1182753,37.571607,126.991806
154,165788,1733838,37.570926,127.001849
...,...,...,...,...
2824,1037376,434322,37.445210,127.156866
2825,372556,156490,37.440918,127.147564
2826,499319,168571,37.437428,127.140722


In [51]:
morning_seoul_in = folium.Map(location = [37.55, 126.98], zoom_start = 12)
HeatMap ( data = morning_joined_data [['geo.latitude', 'geo.longitude', 'people_in']]).add_to(morning_seoul_in)
morning_seoul_in

In [52]:
HeatMap ( data = morning_joined_data [['geo.latitude', 'geo.longitude', 'people_out']]).add_to(morning_seoul_in)
morning_seoul_in

In [53]:
evening_data = data[(pd.to_datetime(data.timestamp).dt.hour > 17) & (pd.to_datetime(data.timestamp).dt.hour < 20)]

In [54]:
evening_station_sum = evening_data.groupby("station_code").sum()

In [55]:
evening_station_sum = evening_station_sum[['people_in', 'people_out']]

In [56]:
evening_station_sum

,people_in,people_out
station_code,,
150,2987183,1343062
151,2034394,269504
152,2940106,609696
153,1688404,518029
154,1592971,357086
...,...,...
2824,361746,726332
2825,137833,329552
2826,135659,313572


In [57]:
evening_joined_data = evening_station_sum.join(station_info)

In [58]:
evening_joined_data

,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,
150,2987183,1343062,37.554648,126.972559
151,2034394,269504,37.564718,126.977108
152,2940106,609696,37.570161,126.982923
153,1688404,518029,37.571607,126.991806
154,1592971,357086,37.570926,127.001849
...,...,...,...,...
2824,361746,726332,37.445210,127.156866
2825,137833,329552,37.440918,127.147564
2826,135659,313572,37.437428,127.140722


In [59]:
evening_seoul_in = folium.Map(location = [37.55, 126.98], zoom_start = 12)
HeatMap ( data = evening_joined_data [['geo.latitude', 'geo.longitude', 'people_in']]).add_to(evening_seoul_in)
evening_seoul_in

In [60]:
HeatMap ( data = evening_joined_data [['geo.latitude', 'geo.longitude', 'people_out']]).add_to(evening_seoul_in)
evening_seoul_in

In [61]:
after_data = data[(pd.to_datetime(data.timestamp).dt.hour == 15)]
after_station_sum = after_data.groupby("station_code").sum()
after_station_sum = after_station_sum[['people_in', 'people_out']]
after_joined_data = after_station_sum.join(station_info)
after_seoul_in = folium.Map(location = [37.55, 126.98], zoom_start = 12)
HeatMap ( data = after_joined_data [['geo.latitude', 'geo.longitude', 'people_in']], gradient={0.3:'blue',0.6:'yellow',1.0:'red'}).add_to(after_seoul_in)
after_seoul_in

In [62]:
HeatMap ( data = after_joined_data [['geo.latitude', 'geo.longitude', 'people_out']], gradient={0.3:'blue',0.6:'yellow',1.0:'red'}).add_to(after_seoul_in)
after_seoul_in